In [1]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 13.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os, optuna
import warnings
from PIL import Image
from tensorflow import keras

from google.colab import drive
drive.mount("/content/drive")

K = keras.backend
# Changing default dir
# os.chdir("/Users/pedroteche/Documents/GitHub/maize-crop-diagnose/")
# optuna.logging.set_verbosity(optuna.logging.WARNING)

Mounted at /content/drive


In [3]:
!mkdir 'data'
!cp -r 'drive/MyDrive/maize-crop-diagnose' 'data'

In [4]:
device_name = tf.test.gpu_device_name()
if device_name != "/device:GPU:0":
    raise SystemError("GPU device not found")
print("Found GPU at: {}".format(device_name))

Found GPU at: /device:GPU:0


In [5]:
class OneCycleScheduler(tf.keras.callbacks.Callback):
    def __init__(
        self,
        iterations,
        max_lr=1e-3,
        start_lr=None,
        start_mom=0.95,
        min_mom=0.85,
        last_iterations=None,
        last_lr=None,
    ):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.start_mom = start_mom
        self.min_mom = min_mom
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr, self.max_lr)
            mom = self._interpolate(
                0, self.half_iteration, self.start_mom, self.min_mom
            )
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(
                self.half_iteration, 2 * self.half_iteration, self.max_lr, self.start_lr
            )
            mom = self._interpolate(
                self.half_iteration,
                2 * self.half_iteration,
                self.min_mom,
                self.start_mom,
            )
        else:
            lr = self._interpolate(
                2 * self.half_iteration, self.iterations, self.start_lr, self.last_lr
            )
            mom = self.start_mom
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)
        K.set_value(self.model.optimizer.momentum, mom)


class OneCycleSchedulerNoMom(tf.keras.callbacks.Callback):
    def __init__(
        self,
        iterations,
        max_lr=1e-3,
        start_lr=None,
        last_iterations=None,
        last_lr=None,
    ):
        self.iterations = iterations
        self.max_lr = max_lr
        self.start_lr = start_lr or max_lr / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_lr = last_lr or self.start_lr / 1000
        self.iteration = 0

    def _interpolate(self, iter1, iter2, lr1, lr2):
        return (lr2 - lr1) * (self.iteration - iter1) / (iter2 - iter1) + lr1

    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            lr = self._interpolate(0, self.half_iteration, self.start_lr, self.max_lr)
        elif self.iteration < 2 * self.half_iteration:
            lr = self._interpolate(
                self.half_iteration, 2 * self.half_iteration, self.max_lr, self.start_lr
            )
        else:
            lr = self._interpolate(
                2 * self.half_iteration, self.iterations, self.start_lr, self.last_lr
            )
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, lr)


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [6]:
IMG_HEIGHT = 64 * 2
IMG_WIDTH = 48 * 2
BATCH_SIZE = 32
EPOCH = 50
TRAIN_DATA_DIR = "data/maize-crop-diagnose/data/train"
TEST_DATA_DIR = "data/maize-crop-diagnose/data/test"
# TRAIN_DATA_DIR = "/Volumes/DOCK-HD/Data/maize-crop-diagnose/train"
# TEST_DATA_DIR = "/Volumes/DOCK-HD/Data/maize-crop-diagnose/test"

In [7]:
train_set, val_set = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DATA_DIR,
    validation_split=0.2,
    subset="both",
    seed=42,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
)
num_classes = len(train_set.class_names)
AUTOTUNE = tf.data.AUTOTUNE
train_set = train_set.prefetch(buffer_size=AUTOTUNE).cache()
val_set = val_set.prefetch(buffer_size=AUTOTUNE).cache()

Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.


In [8]:
TRAIN_SIZE = 11800
TEST_SIZE = 2949

# Optimization

In [10]:
def objective(trial):
    K.clear_session()
    train_set, val_set = tf.keras.utils.image_dataset_from_directory(
        TRAIN_DATA_DIR,
        validation_split=0.2,
        subset="both",
        seed=42,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
    )
    num_classes = len(train_set.class_names)
    AUTOTUNE = tf.data.AUTOTUNE
    train_set = train_set.prefetch(buffer_size=AUTOTUNE).cache()
    val_set = val_set.prefetch(buffer_size=AUTOTUNE).cache()

    # Architecture Hyperparameters
    size_dense = trial.suggest_int("size_dense", 128, 1024, 32)
    activation_function = trial.suggest_categorical("activation_function", ["relu", "elu"])
    # Input layers
    input = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    rescale = keras.layers.Rescaling(1.0 / 255)(input)
    conv_in = keras.layers.Conv2D(
        filters=64, kernel_size=(7, 7), strides=(2, 2), activation=activation_function
    )(rescale)
    pool_in = keras.layers.MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), padding="same"
    )(conv_in)
    # R1
    conv_r1_1 = keras.layers.Conv2D(64, 3, 1, padding="same")(pool_in)
    bn_r1_1 = keras.layers.BatchNormalization()(conv_r1_1)
    if activation_function == "relu":
        relu_r1_1 = keras.layers.ReLU()(bn_r1_1)
    else:
        relu_r1_1 = keras.layers.ELU()(bn_r1_1)
    conv_r1_2 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r1_1)
    bn_r1_2 = keras.layers.BatchNormalization()(conv_r1_2)
    skip_r1 = keras.layers.Add()([bn_r1_2, pool_in])
    if activation_function == "relu":
        relu_r1_2 = keras.layers.ReLU()(skip_r1)
    else:
        relu_r1_2 = keras.layers.ELU()(skip_r1)
    # R2
    conv_r2_1 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r1_2)
    bn_r2_1 = keras.layers.BatchNormalization()(conv_r2_1)
    if activation_function == "relu":
        relu_r2_1 = keras.layers.ReLU()(bn_r2_1)
    else:
        relu_r2_1 = keras.layers.ELU()(bn_r2_1)
    conv_r2_2 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r2_1)
    bn_r2_2 = keras.layers.BatchNormalization()(conv_r2_2)
    skip_r2 = keras.layers.Add()([bn_r2_2, relu_r1_2])
    if activation_function == "relu":
        relu_r2_2 = keras.layers.ReLU()(skip_r2)
    else:
        relu_r2_2 = keras.layers.ELU()(skip_r2)
    # R3
    conv_r3_skip = keras.layers.Conv2D(128, 1, 2, padding="same")(relu_r2_2)
    conv_r3_1 = keras.layers.Conv2D(128, 3, 2, padding="same")(relu_r2_2)
    bn_r3_1 = keras.layers.BatchNormalization()(conv_r3_1)
    if activation_function == "relu":
        relu_r3_1 = keras.layers.ReLU()(bn_r3_1)
    else:
        relu_r3_1 = keras.layers.ELU()(bn_r3_1)
    conv_r3_2 = keras.layers.Conv2D(128, 3, 1, padding="same")(relu_r3_1)
    bn_r3_2 = keras.layers.BatchNormalization()(conv_r3_2)
    skip_r3 = keras.layers.Add()([bn_r3_2, conv_r3_skip])
    if activation_function == "relu":
        relu_r3_2 = keras.layers.ReLU()(skip_r3)
    else:
        relu_r3_2 = keras.layers.ELU()(skip_r3)
    # Output layers
    pool_out = keras.layers.GlobalAveragePooling2D()(relu_r3_2)
    dense_out = keras.layers.Dense(size_dense, activation=activation_function)(pool_out)
    output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)
    # Model
    model = keras.Model(inputs=input, outputs=output)
    # Fitting model
    
    max_lr = trial.suggest_float("max_lr", 0.01, 0.2)
    start_lr_prop = trial.suggest_float("start_lr_prop", 1./50, 1./5)
    last_lr = trial.suggest_float("last_lr", 0.0001, 0.01)
    onecycle = OneCycleSchedulerNoMom(
        TRAIN_SIZE // BATCH_SIZE * EPOCH,
        max_lr=max_lr,
        start_lr=start_lr_prop * max_lr,
        last_lr=last_lr,
    )
    early_stopping = tf.keras.callbacks.EarlyStopping(
        patience=10, restore_best_weights=True, start_from_epoch=20
    )

    model.compile(
        optimizer=tf.keras.optimizers.legacy.SGD(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"],
    )

    model.fit(
        train_set,
        validation_data=val_set,
        epochs=EPOCH,
        batch_size=BATCH_SIZE,
        callbacks=[early_stopping, onecycle],
    )
    # Evaluating and returning F1 score
    loss, acc = model.evaluate(val_set)
    return acc


study = optuna.create_study(
    storage="sqlite:///drive/MyDrive/maize-crop-diagnose/db_maize_models.sqlite3",
    study_name="resnet_onecycle_sgd",
    direction="maximize",
    load_if_exists=True,
)

study.optimize(objective, n_trials=100)
print(f"Best value: {study.best_value} (params: {study.best_params})")

[I 2023-06-09 17:23:17,614] Using an existing study with name 'resnet_onecycle_sgd' instead of creating a new one.


Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
Epoch 1/50
369/369 [==============================] - 23s 35ms/step - loss: 0.3662 - accuracy: 0.8543 - val_loss: 0.9990 - val_accuracy: 0.6877
Epoch 2/50
369/369 [==============================] - 3s 8ms/step - loss: 0.2557 - accuracy: 0.9041 - val_loss: 2.3185 - val_accuracy: 0.5371
Epoch 3/50
369/369 [==============================] - 3s 7ms/step - loss: 0.2049 - accuracy: 0.9237 - val_loss: 0.6251 - val_accuracy: 0.7755
Epoch 4/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1759 - accuracy: 0.9350 - val_loss: 1.1486 - val_accuracy: 0.6094
Epoch 5/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1510 - accuracy: 0.9436 - val_loss: 1.1241 - val_accuracy: 0.6287
Epoch 6/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1332 - accuracy: 0.9510 - val_loss: 1.2988 - val_accuracy: 0.6541
Epoch 7/50
369/369 [===================

[I 2023-06-09 17:25:35,539] Trial 94 finished with value: 0.9833841919898987 and parameters: {'activation_function': 'elu', 'last_lr': 0.0064917785066359104, 'max_lr': 0.11520345992792269, 'size_dense': 576, 'start_lr_prop': 0.13558294803155643}. Best is trial 12 with value: 0.9833841919898987.


Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
Epoch 1/50
369/369 [==============================] - 13s 33ms/step - loss: 0.3764 - accuracy: 0.8502 - val_loss: 0.3061 - val_accuracy: 0.8772
Epoch 2/50
369/369 [==============================] - 3s 8ms/step - loss: 0.2592 - accuracy: 0.9003 - val_loss: 2.2545 - val_accuracy: 0.5527
Epoch 3/50
369/369 [==============================] - 3s 8ms/step - loss: 0.2093 - accuracy: 0.9216 - val_loss: 0.3427 - val_accuracy: 0.8593
Epoch 4/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1776 - accuracy: 0.9353 - val_loss: 0.3991 - val_accuracy: 0.8345
Epoch 5/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1581 - accuracy: 0.9429 - val_loss: 1.8650 - val_accuracy: 0.5398
Epoch 6/50
369/369 [==============================] - 3s 8ms/step - loss: 0.1432 - accuracy: 0.9470 - val_loss: 1.0914 - val_accuracy: 0.5609
Epoch 7/50
369/369 [===================

[I 2023-06-09 17:27:46,824] Trial 95 finished with value: 0.9796541333198547 and parameters: {'activation_function': 'elu', 'last_lr': 0.0005429077109327263, 'max_lr': 0.11235573615612854, 'size_dense': 608, 'start_lr_prop': 0.13651307921592454}. Best is trial 12 with value: 0.9833841919898987.


...

Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
Epoch 1/50
369/369 [==============================] - 14s 35ms/step - loss: 0.3734 - accuracy: 0.8495 - val_loss: 0.5405 - val_accuracy: 0.7806
Epoch 2/50
369/369 [==============================] - 3s 7ms/step - loss: 0.2578 - accuracy: 0.9037 - val_loss: 0.4307 - val_accuracy: 0.8616
Epoch 3/50
369/369 [==============================] - 3s 7ms/step - loss: 0.2095 - accuracy: 0.9226 - val_loss: 0.4504 - val_accuracy: 0.8620
Epoch 4/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1840 - accuracy: 0.9301 - val_loss: 0.5879 - val_accuracy: 0.8203
Epoch 5/50
369/369 [==============================] - 3s 8ms/step - loss: 0.1661 - accuracy: 0.9375 - val_loss: 0.2431 - val_accuracy: 0.9105
Epoch 6/50
369/369 [==============================] - 3s 8ms/step - loss: 0.1519 - accuracy: 0.9430 - val_loss: 3.3455 - val_accuracy: 0.6406
Epoch 7/50
369/369 [===================

[I 2023-06-09 21:05:40,884] Trial 192 finished with value: 0.9877924919128418 and parameters: {'activation_function': 'elu', 'last_lr': 0.00011834844198124078, 'max_lr': 0.06705361956812145, 'size_dense': 704, 'start_lr_prop': 0.12105683863596677}. Best is trial 144 with value: 0.9884706735610962.


Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
Epoch 1/50
369/369 [==============================] - 14s 34ms/step - loss: 0.3782 - accuracy: 0.8501 - val_loss: 0.5176 - val_accuracy: 0.7792
Epoch 2/50
369/369 [==============================] - 3s 8ms/step - loss: 0.2761 - accuracy: 0.8970 - val_loss: 1.5197 - val_accuracy: 0.6924
Epoch 3/50
369/369 [==============================] - 3s 7ms/step - loss: 0.2241 - accuracy: 0.9176 - val_loss: 1.2749 - val_accuracy: 0.6999
Epoch 4/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1966 - accuracy: 0.9271 - val_loss: 0.2116 - val_accuracy: 0.9223
Epoch 5/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1718 - accuracy: 0.9375 - val_loss: 0.2566 - val_accuracy: 0.9125
Epoch 6/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1504 - accuracy: 0.9464 - val_loss: 0.3415 - val_accuracy: 0.8579
Epoch 7/50
369/369 [===================

[I 2023-06-09 21:07:51,643] Trial 193 finished with value: 0.9820278286933899 and parameters: {'activation_function': 'elu', 'last_lr': 0.00018360366574793495, 'max_lr': 0.06862062156447601, 'size_dense': 736, 'start_lr_prop': 0.12175289369324542}. Best is trial 144 with value: 0.9884706735610962.


Best value: 0.9884706735610962 (params: {'activation_function': 'elu', 'last_lr': 0.0014509912873400827, 'max_lr': 0.08406270363816477, 'size_dense': 640, 'start_lr_prop': 0.12643893298902695})


# Test with droput

In [11]:
def objective(trial):
    K.clear_session()
    train_set, val_set = tf.keras.utils.image_dataset_from_directory(
        TRAIN_DATA_DIR,
        validation_split=0.2,
        subset="both",
        seed=42,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
    )
    num_classes = len(train_set.class_names)
    AUTOTUNE = tf.data.AUTOTUNE
    train_set = train_set.prefetch(buffer_size=AUTOTUNE).cache()
    val_set = val_set.prefetch(buffer_size=AUTOTUNE).cache()

    # Architecture Hyperparameters
    size_dense = trial.suggest_int("size_dense", 128, 1024, 32)
    activation_function = trial.suggest_categorical("activation_function", ["relu", "elu"])
    droput_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    # Input layers
    input = keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    rescale = keras.layers.Rescaling(1.0 / 255)(input)
    conv_in = keras.layers.Conv2D(
        filters=64, kernel_size=(7, 7), strides=(2, 2), activation=activation_function
    )(rescale)
    pool_in = keras.layers.MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), padding="same"
    )(conv_in)
    # R1
    conv_r1_1 = keras.layers.Conv2D(64, 3, 1, padding="same")(pool_in)
    bn_r1_1 = keras.layers.BatchNormalization()(conv_r1_1)
    if activation_function == "relu":
        relu_r1_1 = keras.layers.ReLU()(bn_r1_1)
    else:
        relu_r1_1 = keras.layers.ELU()(bn_r1_1)
    conv_r1_2 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r1_1)
    bn_r1_2 = keras.layers.BatchNormalization()(conv_r1_2)
    skip_r1 = keras.layers.Add()([bn_r1_2, pool_in])
    if activation_function == "relu":
        relu_r1_2 = keras.layers.ReLU()(skip_r1)
    else:
        relu_r1_2 = keras.layers.ELU()(skip_r1)
    # R2
    conv_r2_1 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r1_2)
    bn_r2_1 = keras.layers.BatchNormalization()(conv_r2_1)
    if activation_function == "relu":
        relu_r2_1 = keras.layers.ReLU()(bn_r2_1)
    else:
        relu_r2_1 = keras.layers.ELU()(bn_r2_1)
    conv_r2_2 = keras.layers.Conv2D(64, 3, 1, padding="same")(relu_r2_1)
    bn_r2_2 = keras.layers.BatchNormalization()(conv_r2_2)
    skip_r2 = keras.layers.Add()([bn_r2_2, relu_r1_2])
    if activation_function == "relu":
        relu_r2_2 = keras.layers.ReLU()(skip_r2)
    else:
        relu_r2_2 = keras.layers.ELU()(skip_r2)
    # R3
    conv_r3_skip = keras.layers.Conv2D(128, 1, 2, padding="same")(relu_r2_2)
    conv_r3_1 = keras.layers.Conv2D(128, 3, 2, padding="same")(relu_r2_2)
    bn_r3_1 = keras.layers.BatchNormalization()(conv_r3_1)
    if activation_function == "relu":
        relu_r3_1 = keras.layers.ReLU()(bn_r3_1)
    else:
        relu_r3_1 = keras.layers.ELU()(bn_r3_1)
    conv_r3_2 = keras.layers.Conv2D(128, 3, 1, padding="same")(relu_r3_1)
    bn_r3_2 = keras.layers.BatchNormalization()(conv_r3_2)
    skip_r3 = keras.layers.Add()([bn_r3_2, conv_r3_skip])
    if activation_function == "relu":
        relu_r3_2 = keras.layers.ReLU()(skip_r3)
    else:
        relu_r3_2 = keras.layers.ELU()(skip_r3)
    # Output layers
    pool_out = keras.layers.GlobalAveragePooling2D()(relu_r3_2)
    dense_out = keras.layers.Dense(size_dense, activation=activation_function)(pool_out)
    dropout = keras.layers.Dropout(droput_rate)
    output = keras.layers.Dense(num_classes, activation="softmax")(dense_out)
    # Model
    model = keras.Model(inputs=input, outputs=output)
    # Fitting model
    
    max_lr = trial.suggest_float("max_lr", 0.01, 0.2)
    start_lr_prop = trial.suggest_float("start_lr_prop", 1./50, 1./5)
    last_lr = trial.suggest_float("last_lr", 0.0001, 0.01)
    onecycle = OneCycleSchedulerNoMom(
        TRAIN_SIZE // BATCH_SIZE * EPOCH,
        max_lr=max_lr,
        start_lr=start_lr_prop * max_lr,
        last_lr=last_lr,
    )
    early_stopping = tf.keras.callbacks.EarlyStopping(
        patience=10, restore_best_weights=True, start_from_epoch=20
    )

    model.compile(
        optimizer=tf.keras.optimizers.legacy.SGD(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=["accuracy"],
    )

    model.fit(
        train_set,
        validation_data=val_set,
        epochs=EPOCH,
        batch_size=BATCH_SIZE,
        callbacks=[early_stopping, onecycle],
    )
    # Evaluating and returning F1 score
    loss, acc = model.evaluate(val_set)
    return acc


study = optuna.create_study(
    storage="sqlite:///drive/MyDrive/maize-crop-diagnose/db_maize_models.sqlite3",
    study_name="resnet_onecycle_drop_sgd",
    direction="maximize",
    load_if_exists=True,
)

study.optimize(objective, n_trials=100)
print(f"Best value: {study.best_value} (params: {study.best_params})")

[I 2023-06-09 21:41:12,701] A new study created in RDB with name: resnet_onecycle_drop_sgd


Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
Epoch 1/50
369/369 [==============================] - 14s 34ms/step - loss: 0.4239 - accuracy: 0.8356 - val_loss: 1.2305 - val_accuracy: 0.4788
Epoch 2/50
369/369 [==============================] - 3s 8ms/step - loss: 0.2732 - accuracy: 0.8972 - val_loss: 1.1105 - val_accuracy: 0.4924
Epoch 3/50
369/369 [==============================] - 3s 8ms/step - loss: 0.1905 - accuracy: 0.9300 - val_loss: 1.4013 - val_accuracy: 0.6409
Epoch 4/50
369/369 [==============================] - 3s 8ms/step - loss: 0.1498 - accuracy: 0.9480 - val_loss: 1.2053 - val_accuracy: 0.6260
Epoch 5/50
369/369 [==============================] - 3s 8ms/step - loss: 0.1259 - accuracy: 0.9557 - val_loss: 1.6492 - val_accuracy: 0.5524
Epoch 6/50
369/369 [==============================] - 3s 8ms/step - loss: 0.1153 - accuracy: 0.9597 - val_loss: 1.2906 - val_accuracy: 0.5738
Epoch 7/50
369/369 [===================

[I 2023-06-09 21:42:54,008] Trial 0 finished with value: 0.9674465656280518 and parameters: {'activation_function': 'relu', 'dropout_rate': 0.26815512554163656, 'last_lr': 0.0072425649844705745, 'max_lr': 0.1960650632386013, 'size_dense': 480, 'start_lr_prop': 0.18811920831274184}. Best is trial 0 with value: 0.9674465656280518.


Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
Epoch 1/50
369/369 [==============================] - 13s 33ms/step - loss: 0.3669 - accuracy: 0.8526 - val_loss: 0.4698 - val_accuracy: 0.8118
Epoch 2/50
369/369 [==============================] - 3s 7ms/step - loss: 0.2543 - accuracy: 0.9036 - val_loss: 1.6386 - val_accuracy: 0.6870
Epoch 3/50
369/369 [==============================] - 3s 7ms/step - loss: 0.2089 - accuracy: 0.9210 - val_loss: 0.7253 - val_accuracy: 0.6802
Epoch 4/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1842 - accuracy: 0.9316 - val_loss: 0.6141 - val_accuracy: 0.6979
Epoch 5/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1586 - accuracy: 0.9418 - val_loss: 1.4783 - val_accuracy: 0.5829
Epoch 6/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1460 - accuracy: 0.9475 - val_loss: 2.2288 - val_accuracy: 0.6029
Epoch 7/50
369/369 [===================

[I 2023-06-09 21:45:23,024] Trial 1 finished with value: 0.9877924919128418 and parameters: {'activation_function': 'elu', 'dropout_rate': 0.42890523677244097, 'last_lr': 0.003692282680741792, 'max_lr': 0.0907141221879276, 'size_dense': 192, 'start_lr_prop': 0.1442133865351896}. Best is trial 1 with value: 0.9877924919128418.


...

Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
Epoch 1/50
369/369 [==============================] - 14s 34ms/step - loss: 0.3656 - accuracy: 0.8590 - val_loss: 1.2178 - val_accuracy: 0.5229
Epoch 2/50
369/369 [==============================] - 3s 7ms/step - loss: 0.2292 - accuracy: 0.9135 - val_loss: 1.1570 - val_accuracy: 0.6311
Epoch 3/50
369/369 [==============================] - 3s 7ms/step - loss: 0.2013 - accuracy: 0.9271 - val_loss: 0.4680 - val_accuracy: 0.8182
Epoch 4/50
369/369 [==============================] - 3s 8ms/step - loss: 0.1739 - accuracy: 0.9339 - val_loss: 1.0976 - val_accuracy: 0.6612
Epoch 5/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1471 - accuracy: 0.9465 - val_loss: 1.0423 - val_accuracy: 0.6575
Epoch 6/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1287 - accuracy: 0.9528 - val_loss: 0.3044 - val_accuracy: 0.8840
Epoch 7/50
369/369 [===================

[I 2023-06-10 01:19:00,770] Trial 98 finished with value: 0.9793150424957275 and parameters: {'activation_function': 'relu', 'dropout_rate': 0.2164240675207795, 'last_lr': 0.0010961672167798902, 'max_lr': 0.08734783170991406, 'size_dense': 448, 'start_lr_prop': 0.19340513618253186}. Best is trial 40 with value: 0.988131582736969.


Found 14749 files belonging to 3 classes.
Using 11800 files for training.
Using 2949 files for validation.
Epoch 1/50
369/369 [==============================] - 14s 34ms/step - loss: 0.3679 - accuracy: 0.8519 - val_loss: 0.5353 - val_accuracy: 0.7854
Epoch 2/50
369/369 [==============================] - 3s 7ms/step - loss: 0.2583 - accuracy: 0.9036 - val_loss: 1.0588 - val_accuracy: 0.7525
Epoch 3/50
369/369 [==============================] - 3s 7ms/step - loss: 0.2162 - accuracy: 0.9181 - val_loss: 0.8257 - val_accuracy: 0.7786
Epoch 4/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1919 - accuracy: 0.9264 - val_loss: 0.4686 - val_accuracy: 0.8406
Epoch 5/50
369/369 [==============================] - 3s 7ms/step - loss: 0.1746 - accuracy: 0.9347 - val_loss: 0.2839 - val_accuracy: 0.8864
Epoch 6/50
369/369 [==============================] - 3s 8ms/step - loss: 0.1595 - accuracy: 0.9421 - val_loss: 0.6157 - val_accuracy: 0.7820
Epoch 7/50
369/369 [===================

[I 2023-06-10 01:20:41,629] Trial 99 finished with value: 0.5293319821357727 and parameters: {'activation_function': 'elu', 'dropout_rate': 0.32043450225426034, 'last_lr': 0.0016605899824788729, 'max_lr': 0.05958262855465033, 'size_dense': 864, 'start_lr_prop': 0.12767826535598314}. Best is trial 40 with value: 0.988131582736969.


Best value: 0.988131582736969 (params: {'activation_function': 'elu', 'dropout_rate': 0.24596124523495363, 'last_lr': 0.0008319884847908514, 'max_lr': 0.09611978174008246, 'size_dense': 256, 'start_lr_prop': 0.13723620116968183})
